In [1]:
!pip install transformers torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [5]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Load the LegalBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModelForQuestionAnswering.from_pretrained("nlpaueb/legal-bert-base-uncased")

# Function to perform question-answering
def answer_question(question, context):
    # Tokenize input question and context
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True)

    # Get model outputs
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the most likely beginning and end of the answer
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1

    # Convert tokens back to string
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

    return answer


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Example context and question
context = """
In the case of Roe v. Wade, the U.S. Supreme Court ruled that a state law that banned abortions except to save the life of the mother was unconstitutional.
The court ruled that states were forbidden from outlawing or regulating any aspect of abortion performed during the first trimester of pregnancy.
"""
question = "What did the U.S. Supreme Court rule in Roe v. Wade?"

# Get the answer from LegalBERT
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer}")


Question: What did the U.S. Supreme Court rule in Roe v. Wade?
Answer: 


In [7]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Load the LegalBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModelForQuestionAnswering.from_pretrained("nlpaueb/legal-bert-base-uncased")

# Function to perform question-answering
def answer_question(question, context):
    # Tokenize input question and context
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True)

    # Get model outputs
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the start and end logits
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Find the best start and end index (highest logits)
    answer_start = torch.argmax(start_logits)
    answer_end = torch.argmax(end_logits) + 1

    # Check if the model is predicting any valid span
    if answer_start > answer_end or answer_start == 0 or answer_end == 0:
        return "No valid answer found."

    # Convert tokens back to string
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

    # Return the answer
    return answer.strip()

# Example context and question
context = """
In the case of Roe v. Wade, the U.S. Supreme Court ruled that a state law that banned abortions except to save the life of the mother was unconstitutional.
The court ruled that states were forbidden from outlawing or regulating any aspect of abortion performed during the first trimester of pregnancy.
"""
question = "What did the U.S. Supreme Court rule in Roe v. Wade?"

# Get the answer from LegalBERT
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer}")


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question: What did the U.S. Supreme Court rule in Roe v. Wade?
Answer: . s. supreme court rule in roe v. wade? [SEP] in the case of roe v. wade, the u. s. supreme court ruled that a state law that banned abortions except to save the life of the mother was unconstitutional. the
